# Pruebas para la generacion de datos de entrenamiento

## Cambios para que funcione con clases

Para exportar se ve que crear una clase para almacenar todas las percepciones es muy util por lo que se procede a modificar todas las funciones anteriores que lo requieran para que funcionen con la clase de percepciones.

#### Librerias

In [36]:
import os
import math
import pickle
from zmqRemoteApi import RemoteAPIClient
from random import randint as ri
from random import uniform as ru

#### Creacion de clases

In [66]:
#Class for simulation object
class sim_objects ():
    def __init__(self):
        self.client = RemoteAPIClient()
        self.sim = self.client.getObject('sim')
        self.file = 0
        self.random_increments = []
        self.executed = False
        self.joint_handler_ids = []
        self.num_joints = 0
        self.obj_handler_ids = []
        self.sequencies = 1
        self.seq_steps = 2
        self.training_data = []
        
sim_obj = sim_objects()

# Class to store the different handlers the robot needs to operate in the simulator. 
class joint_handlers_class ():
    def __init__(self):
        self.JRF0 = 0
        self.JRF1 = 0
        self.JRB0 = 0
        self.JRB1 = 0
        self.JLF0 = 0
        self.JLF1 = 0
        self.JLB0 = 0
        self.JLB1 = 0
        
handler = joint_handlers_class()

class obj_handlers_class ():
    def __init__(self):
        self.JRF2 = 0
        self.JRB2 = 0
        self.JLF2 = 0
        self.JLB2 = 0
        self.base = 0

obj_handler = obj_handlers_class()

#### Funciones de carga del quad y generacion de acciones random

In [61]:
def init_position():
    start_pos = ri(0,1)
    #Fixes the last joints position to 0 so they wont move
    initial_obj_positions = [0,0,0,0]
    for i in range(len(initial_obj_positions)):
        sim_obj.sim.setJointTargetPosition((sim_obj.obj_handler_ids[i]), initial_obj_positions[i])
    if start_pos == 0:
        initial_joint_positions = [0,0,0,0,0,0,0,0]
        #Set all the joints to their position
        for i in range(len(initial_joint_positions)):
            sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], initial_joint_positions[i])
        #After setting all the joints it moves    
        for i in range(20):
            sim_obj.client.step()
    
            

In [39]:
# Function that load the robot in the scene at the beggining of each evaluation.
def load_quad_class():
    # Opening the connection with CoppeliaSim simulator
    sim_obj.client = RemoteAPIClient()
    sim_obj.sim = sim_obj.client.getObject('sim')
    
    # Setting the simulation mode for working in sinchronous mode. The simulation timming is controlled by the Python program and not by the simulation itselfs.
    sim_obj.client.setStepping(True)
    
    #Right-front
    handler.JRF0 = sim_obj.sim.getObject("/JRF0")
    handler.JRF1 = sim_obj.sim.getObject("/JRF1")
    #Right-back
    handler.JRB0 = sim_obj.sim.getObject("/JRB0")
    handler.JRB1 = sim_obj.sim.getObject("/JRB1")
    #Left-front
    handler.JLF0 = sim_obj.sim.getObject("/JLF0")
    handler.JLF1 = sim_obj.sim.getObject("/JLF1")
    #Left-back
    handler.JLB0 = sim_obj.sim.getObject("/JLB0")
    handler.JLB1 = sim_obj.sim.getObject("/JLB1")

    #Setting obj handlers
    obj_handler.base = sim_obj.sim.getObject("/Base")
    obj_handler.JRF2 = sim_obj.sim.getObject("/JRF2")
    obj_handler.JRB2 = sim_obj.sim.getObject("/JRB2")
    obj_handler.JLF2 = sim_obj.sim.getObject("/JLF2")
    obj_handler.JLB2 = sim_obj.sim.getObject("/JLB2")

    
    #Stores the different id for every handler starting with the base
    sim_obj.joint_handler_ids = list(handler.__dict__.values())
    print(sim_obj.joint_handler_ids)
    sim_obj.num_joints = len(sim_obj.joint_handler_ids)
    print(sim_obj.num_joints)
    sim_obj.obj_handler_ids = list(obj_handler.__dict__.values())
    print(sim_obj.obj_handler_ids)

    sim_obj.sim.startSimulation()

In [40]:
#Generate list of random positions
def rand_gen():
    randomlist = []
    for i in range(sim_obj.num_joints):
        n = ru(-10, 10) * math.pi / 180
        randomlist.append(n)
    sim_obj.random_increments.append(randomlist)

#### Clase y funcion para percepciones

In [67]:

#Class to store all the perceptions
class perceptions ():
    def __init__(self):
        #Sequence
        self.sequence = 0

        #Initial perceptions
        self.prev_j_positions = []
        self.prev_j_velocities = []
        self.prev_j_forces = []
        self.prev_base_pos_x = []
        self.prev_base_pos_y = []
        self.prev_base_pos_z = []
        self.prev_base_ori_alpha = []
        self.prev_base_ori_beta = []
        self.prev_base_ori_gamma = []
        self.prev_base_accel = []

        #Resulting perceptions
        self.post_j_positions = []
        self.post_j_velocities = []
        self.post_j_forces = []
        self.post_base_pos_x = []
        self.post_base_pos_y = []
        self.post_base_pos_z = []
        self.post_base_ori_alpha = []
        self.post_base_ori_beta = []
        self.post_base_ori_gamma = []
        self.post_base_accel = []

perception = perceptions()

In [52]:
def move_joints():
    for i in range(sim_obj.num_joints):
        sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
    #After setting all the joints it moves
    for i in range(4):
        sim_obj.client.step()

In [68]:

def get_joints_preceptions():
    positions = []
    forces = []
    velocities = []
    base_x, base_y,base_z = sim_obj.sim.getObjectPosition(obj_handler.base, sim_obj.sim.handle_world)
    base_alpha, base_beta, base_gamma = sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world)
    base_accel = [0,0,0] #sim_obj.sim.getObjectOrientation(obj_handler.base, sim_obj.sim.handle_world) --- MISSING
    for i in sim_obj.joint_handler_ids:
        #Angular/linear position of the joint
        pos = sim_obj.sim.getJointPosition(i)
        positions.append(pos)
        #Torque/force being aplied by the joint
        force = sim_obj.sim.getJointForce(i)
        forces.append(force)
        #Angular/linear velocity of the joint's movement
        veloc = sim_obj.sim.getJointVelocity(i)
        velocities.append(veloc)
    if  not sim_obj.executed:
        print("Getting prev perceptions")
        perception.prev_j_positions.append(positions)
        perception.prev_j_velocities.append(velocities)
        perception.prev_j_forces.append(forces)
        perception.prev_base_pos_x.append(base_x)
        perception.prev_base_pos_y.append(base_y)
        perception.prev_base_pos_z.append(base_z)
        perception.prev_base_ori_alpha.append(base_alpha)
        perception.prev_base_ori_beta.append(base_beta)
        perception.prev_base_ori_gamma.append(base_gamma)
        perception.prev_base_accel.append(base_accel)
    else:
        print("Getting post perceptions")
        perception.post_j_positions.append(positions)
        perception.post_j_velocities.append(velocities)
        perception.post_j_forces.append(forces)
        perception.post_base_pos_x.append(base_x)
        perception.post_base_pos_y.append(base_y)
        perception.post_base_pos_z.append(base_z)
        perception.post_base_ori_alpha.append(base_alpha)
        perception.post_base_ori_beta.append(base_beta)
        perception.post_base_ori_gamma.append(base_gamma)
        perception.post_base_accel.append(base_accel)

#### Exportar

In [62]:
def export_pickle():    
    #Dump data in file
    sim_obj.file = open(os.getcwd() + r'\training_dataset.txt', 'wb')
    pickle.dump(perception, sim_obj.file)
    sim_obj.file.close()

#### Ejecucion

##### Una ejecucion y un paso (una accion por modulo)

In [64]:
#Main function for one execution
def main():
    print("main function")
    sim_obj.sim.startSimulation()
    load_quad_class()
    rand_gen()
    get_joints_preceptions()
    #Execute every joint action
    print("Executing actions")
    for i in range(len(sim_obj.random_increments[-1])):
        sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
        sim_obj.client.step()
        sim_obj.client.step()
        sim_obj.client.step()
        sim_obj.client.step()
    sim_obj.executed = True
    get_joints_preceptions()
    sim_obj.executed = False
    sim_obj.sim.stopSimulation()
    #export()

In [ ]:
#main()

print("")

print("Handler ids: ")
print(sim_obj.joint_handler_ids)
print("Joints handler ids: ")
print(sim_obj.joint_handler_ids)

print("")

data = 0
num_exec = []
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
    num_exec.append(len(perception.__dict__[i]))
print(f"Cantidad total de datos para {max(num_exec)} ejecuciones con 1 paso por ejecucion: {data}")
print(f"Datos por ejecucion: {data//max(num_exec)}")

##### Una secuencia de multiples pasos (una accion por modulo)

In [ ]:
sim_obj.sequencies = 2
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 2
print("Steps por secuencia: ",sim_obj.seq_steps)
perception = perceptions()

In [ ]:
#Main function for one execution and multiple steps
def main():
    k = 0
    print("main function")
    sim_obj.sim.startSimulation()
    load_quad_class()
    for j in range(sim_obj.seq_steps):
        print("----------- Secuencia: ", k)
        print("----------- Paso: ", j)
        rand_gen()
        get_joints_preceptions()
        #Execute every joint action
        print(perception.prev_j_positions[-1])
        print("Executing actions")
        for i in range(len(sim_obj.random_increments[-1])):
            sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
            sim_obj.client.step()
            sim_obj.client.step()
            sim_obj.client.step()
            sim_obj.client.step()
        sim_obj.executed = True
        get_joints_preceptions()
        sim_obj.executed = False
        print("")
    print("Simulation has finished")
    sim_obj.sim.stopSimulation()
    #export()

In [ ]:
#if __name__ == "__main__":
#    main()

In [ ]:
sim_obj.sim.stopSimulation()
main()

print("")
data = 0
num_exec = []
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
    num_exec.append(len(perception.__dict__[i]))
print(f"Cantidad total de datos: {data}")

##### Multiples secuencias y multiples pasos

In [32]:
sim_obj.sequencies = 2
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 3
print("Steps por secuencia: ",sim_obj.seq_steps)
perception = perceptions()

#Main function for multiple executions and multiple steps
def main():
    print("main function")
    for k in range(sim_obj.sequencies):
        print(f"{k} sequence has started")
        print("")
        load_quad_class()
        init_position()
        for j in range(sim_obj.seq_steps):
            print("----------- Secuencia: ", k)
            print("----------- Paso: ", j)
            rand_gen()
            get_joints_preceptions()
            #Execute every joint action
            print("Executing actions")
            for i in range(sim_obj.num_joints):
                sim_obj.sim.setJointTargetPosition(sim_obj.joint_handler_ids[i], (sim_obj.random_increments[-1][i]+perception.prev_j_positions[-1][i]))
                sim_obj.client.step()
                sim_obj.client.step()
                sim_obj.client.step()
                sim_obj.client.step()
            sim_obj.executed = True
            get_joints_preceptions()
            sim_obj.executed = False
            print("")
        print(f"{k} simulation has finished")
        print("")
        sim_obj.sim.stopSimulation()
    #export()

Numero de secuencias:  2
Steps por secuencia:  3


In [ ]:
main()

In [ ]:
data = 0
num_exec = []
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
    num_exec.append(len(perception.__dict__[i]))
print(f"Cantidad total de datos para {max(num_exec)} secuencias con 1 paso por ejecucion: {data}")
print(f"Datos por ejecucion: {data//max(num_exec)}")

In [ ]:
data = 0
for i in perception.__dict__.keys():
    #print(i)
    for k in perception.__dict__[i]:
        #print(k)
        #print("")
        data = data + len(k)
print(data)

##### Pruebas con la funcion de exportar

In [ ]:
sim_obj.file = open(os.getcwd() + r'\test.txt', 'wb')
pickle.dump(perception, sim_obj.file)
sim_obj.file.close()
for i in perception.__dict__.keys():
    print(i)
    for k in perception.__dict__[i]:
        print(k)
        print("")

##### Pruebas para la carga de datos

In [ ]:
load_file = open(os.getcwd() + r'\test.txt', 'rb')
pickled_prception_class = pickle.load(load_file)
load_file.close()
for i in pickled_prception_class.__dict__.keys():
    print(i)
    for k in pickled_prception_class.__dict__[i]:
        print(k)
        print("")

In [ ]:
if perception.__dict__ == pickled_prception_class.__dict__:
    print("Se conserva el mismo objeto")
else:
    print("No se conserva el mismo objeto")

##### Pruebas finales con el main

In [56]:
sim_obj.sequencies = 2
print("Numero de secuencias: ",sim_obj.sequencies)
sim_obj.seq_steps = 10
print("Steps por secuencia: ",sim_obj.seq_steps)
perception = perceptions()

#Main function for multiple executions and multiple steps
def main():
    print("main function")
    for k in range(sim_obj.sequencies):
        print(f"{k} sequence has started")
        print("")
        load_quad_class()
        init_position()
        for j in range(sim_obj.seq_steps):
            print("----------- Secuencia: ", k)
            print("----------- Paso: ", j)
            rand_gen()
            get_joints_preceptions()
            #Execute every joint action
            print("Executing actions")
            move_joints()
            sim_obj.executed = True
            get_joints_preceptions()
            sim_obj.executed = False
            print("")
        print(f"{k} simulation has finished")
        print("")
        sim_obj.sim.stopSimulation()
    #export()

Numero de secuencias:  2
Steps por secuencia:  10


In [59]:
main()

main function
0 sequence has started

[25, 29, 37, 41, 49, 53, 61, 65]
8
[33, 45, 57, 69, 22]
----------- Secuencia:  0
----------- Paso:  0
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  1
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  2
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  3
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  4
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  5
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  6
Getting prev perceptions
Executing actions
Getting post perceptions

----------- Secuencia:  0
----------- Paso:  7
Getting prev perceptions
Executing actions
Gett

##### Pruebas con pandas